# Programmer's Portfolio / Project Documentation

## A. Documentation

### Core Functions & Main Components

This project is a **Design Thinking Bot** application built with Next.js. The core functionality revolves around managing design thinking projects through various phases (Empathize, Define, Ideate, Prototype, Test) and integrating AI assistance.

#### 1. AI Chat Logic (`src/app/api/chat/route.js`)
This is the intelligence core of the application.
- **Functionality**: Handles conversations between the user and the "Socratic Bot".
- **Mechanism**: 
  - It receives a user message and the current Project Phase.
  - It constructs a **Phase-Specific Context** using a predefined `QUESTION_BANK`. For example, if the user is in the "Empathize" phase, the bot is instructed to ask about user feelings and observations.
  - It builds a prompt that includes the System Prompt (Socratic method instructions), Phase Context, and Conversation History.
  - It calls the **Google Gemini API** (`gemini-2.5-flash` model) to generate a response.

#### 2. Project Frontend Logic (`src/app/project/page.jsx`)
The main interactive interface for the student.
- **State Management**: Uses React `useState` to track the current `phase` (e.g., Empathize, Define), the list of `messages` (chat history), and `stageData` (the artifacts created by the user).
- **Phase Transition**: The `changePhase` function handles moving between design thinking stages. It triggers a save of the current work (`saveIdeationState`) before switching contexts.
- **Real-time Updates**: Function `ProjectContent.sendMessage` sends user input to the backend and updates the UI optimistically while waiting for the AI response.
- **Data Persistence**: Regularly syncs state with the backend via `saveIdeationState`, which posts to `/api/projects`.

#### 3. Authentication System (`src/app/api/login/route.js` & `src/app/api/users/route.js`)
Handles user identity and access control.
- **Registration (`/api/users`)**: Checks if a user already exists and creates a new `User` document in MongoDB. Supports roles like 'student' and 'teacher'.
- **Login (`/api/login`)**: Verifies credentials. Currently implemented as a direct comparison for simplicity in this version, but structured to support hashing.
- **Context**: The frontend identifies the logged-in user and passes this identity via headers (`x-current-user`) to protected API routes.

#### 4. Database Connection (`src/utils/db.js`)
The backbone of data persistence.
- **Purpose**: Ensures efficient database connections in a serverless/Next.js environment.
- **Mechanism**: Utilizes a caching strategy (`global.mongoose`) to reuse connections across hot reloads, preventing connection leaks.

#### 5. Project Data Modeling (`src/models/Project.js`)
The central data structure schema.
- **Phases**: Defines strict schemas for each design phase, ensuring artifacts like 'empathyMap' or 'reframedProblem' are stored correctly.
- **Collaboration**: Includes `sharedWith` fields to allow multiple users (students/teachers) to access the same project.
  
---

## B. Technical Specifications

### 1. APIs utilized
The project utilizes the following internal API endpoints:
- **`/api/projects`**: Core project management (Create, Read, Update).
- **`/api/chat`**: AI Chat interface interacting with Google Gemini.
- **`/api/users`**: User management (Registration, fetching users by role).
- **`/api/login`**: Authentication endpoint.
- **Cloudinary API**: Used for storing and retrieving user-uploaded images and files.
- **Google Generative AI SDK**: Used for AI text generation.

### 2. Database
**System**: MongoDB
**Link/Connection**: The connection string is managed via the `MONGODB_URI` environment variable. 
> Security Note: The actual database URI is credential-protected and not exposed in the client-side code.

### 3. Environments & Dependencies
**Core Environment**:
- **Runtime**: Node.js
- **Framework**: Next.js 16.0.10
- **Library**: React 19.2.1

**Key Dependencies**:
- **`mongoose` (^9.0.2)**: ODM for MongoDB.
- **`@google/generative-ai` (^0.24.1)**: AI Model integration.
- **`cloudinary` (^2.8.0)**: Media handling.
- **`tailwindcss` (^4)**: Styling framework.
- **`react-to-print`**: PDF generation.

### 4. External Code
**Database Connection Pattern**:
The caching logic in `src/utils/db.js` is a standard pattern for Next.js to avoid connection exhaustion.
```javascript
// src/utils/db.js
let cached = global.mongoose;
if (!cached) {
    cached = global.mongoose = { conn: null, promise: null };
}
```
*Reference*: Adapted from standard Next.js + Mongoose integration patterns.

---

## C. Developer Guide & Architecture

### 1. Architecture Overview
The application follows the **Client-Server** architecture typical of Next.js 14+ specific applications.
- **Server-Side**: API routes (`src/app/api/*`) handle business logic, database interactions, and AI service orchestration. These ensure that sensitive keys (like `GEMINI_API_KEY` and `MONGODB_URI`) remain secure on the server.
- **Client-Side**: React components (`src/components/*`) and Pages (`src/app/*`) handle user interaction and state. The application uses client-side rendering (`'use client'`) for interactive dashboards like the `BrainstormingCanvas`.

### 2. Component Hierarchy
Key complex components driving the User Experience:

- **`BrainstormingCanvas.jsx`**: An interactive canvas allowing users to place, move, and organize ideas visually. Logic involves coordinate mapping and collision detection for UI elements.
- **`PrioritizationMatrix.jsx`**: A specialized grid component for sorting ideas based on `Impact` vs `Effort`. It helps users filter brainstormed ideas into actionable categories.
- **`SketchPad.jsx`**: A drawing tool integrated into the Prototyping phase, enabling users to create rough sketches directly in the browser.
- **`TechSpecGenerator.jsx`**: A tool that likely takes selected ideas and prompts the user to define technical constraints and requirements, bridging the gap between Ideation and prototyping.

### 3. Setup & Development Workflow
To run this project locally:
1. **Environment Setup**: Create a `.env.local` file with:
   - `MONGODB_URI`: Connection string to MongoDB Atlas.
   - `GEMINI_API_KEY`: API key from Google AI Studio.
   - `CLOUDINARY_*`: Keys for media storage.
2. **Install Dependencies**: Run `npm install` to hydrate `node_modules`.
3. **Run Development Server**: execute `npm run dev`. The app will start at `http://localhost:3000`.

### 4. Future Roadmap / TODOs
- **Auth Hardening**: Current login is a direct string comparison. Future version should implement `bcrypt` for password hashing.
- **Middleware**: Add `middleware.js` to protect `/project` routes server-side instead of relying just on client-side redirects.
- **Real-time Collaboration**: The schema supports `sharedWith`, but real-time socket connections (e.g., Socket.io) could be added to see teammate cursor movements on the `BrainstormingCanvas`.

---

## D. Implementation Guide: Making Changes

This section guides developers on how to extend the project's core capabilities.

### 1. How to Add a New Design Phase
Currently, the project uses 5 phases (Empathize, Define, Ideate, Prototype, Test). To add a 6th phase (e.g., "Launch"):
1. **Update Data Model**: Modify `src/models/Project.js`. Add the new phase to the `enum` definition and create a new key in `stageData` with its specific schema/fields.
2. **Update Frontend Map**: In `src/app/project/page.jsx`, update the `PHASE_INFO` object to include the new phase key (e.g., `'Launch': { tools: [...], capabilities: ... }`).
3. **Update AI Context**: In `src/app/api/chat/route.js`, add a new key to `QUESTION_BANK` with Socratic questions relevant to the new phase. The AI will automatically pick this up when the user enters the phase.

### 2. How to Add a New Tool/Component
To add a new interactive tool (e.g., "User Journey Map"):
1. **Create Component**: Build the component in `src/components/YourNewTool.jsx`.
2. **Integrate in Main Page**: Import it into `src/app/project/page.jsx`.
3. **Conditional Rendering**: Add a condition in the `renderStageContent` function (or equivalent switch case) to render your component only when `activePhase === 'YourPhase'`.
4. **State Persistence**: Ensure the component calls a handler (like `handleSave`) that updates the parent `stageData` state, so changes are saved to MongoDB via `saveIdeationState`.

### 3. How to Modify AI Logic
To change how the bot behaves:
- **System Prompt**: Edit `SYSTEM_PROMPT` in `src/app/api/chat/route.js`. This controls the bot's persona (Socratic Coach vs. Direct Answerer).
- **Context Injection**: To give the bot more awareness (e.g., knowing the user's name), update the `fullPrompt` construction line to interpolate new variables.

---

## E. Prompts that where used
1. we have an export pdf functions, i want you to remake this pdf coz this function was built on an earlier version, you choose which details to add to the pdf, it will include details from all of the phases that were already filled, it can be longer than one page, tell me if you have quesitons before you start

2. go through each phase, and in each one, inside all of the there are funtions and things to do, but most of them are white and dont fit our space theme, change the colours so it will fit, go phase by phase and change 

3. just created a new project, moved to a certain phase, logged out the nlogged in and opened it again and it opened the first phase instead of the phase that we go to in the first session. fix it

4. make it so inside the alarm message when trying to go to the next phase, the alamr will also include a message saying something about the unchecked items

5. inside all the projects in the dashboard, it says Empathize tasks: Checked?
regardless of the project.
also  make the dashboard more appealing, add more things to visualize, maybe precentage of completion or summery of bot conversations and number of replys with the bot. also each project will be in the color of the phase it is in, choose the colors to fit the colors that are inside the projects already for each phase. if you have questions ask me

---

## F. Users inside the database
**Teacher:**

username: teach

password: 123

**User:**

username: rafi

password: 123456